# OSD JSON - Construction

JSON Format. Will stress more important information

```js
// Internal Nodes
{
    "name": "room2",
    "children": [ {...} ],
    "type": // "host",  "ipservice",  "osd",  "rack",  "room",  "root",  "row"
    "data": the raw data
}

// OSD Nodes
{
    "name": "osd.323",
    "children": [ {...} ],
    "type": "osd", // certainly osd
    "exists": 1,   // 0, 1
    "status": "up" ,// "up", "down
    "data": the raw data
}
```

## Production

- Source json directory: `rawdata`
- Product json directory: `product`

In [18]:
import json
import glob as glob
from copy import deepcopy

In [2]:
def indexableNodes(nodes):
    return {str(node["id"]):node for node in nodes}

def getRoots(nodes):
    ROOT_TYPE = "root" # ROOT_ID = 10
    return [ k for k in nodes if nodes[k]["type"] == ROOT_TYPE]

### OSD and Internal Node Generation

In [39]:
def getOSDNode(node):
    osd_health = 1 if node["status"] == "up" else 0
    ret = { 
        "name": node["name"], 
        "status": node["status"], # up/down
        "type": node["type"],
        "exists": node["exists"],   # 
        "osd_counts": 1,
        "osd_health": osd_health,
        "data": node
    }
    return ret

def getInternalNode(node, cnodes):
    osd_counts = sum([n["osd_counts"] for n in cnodes])
    osd_health = sum([n["osd_health"] for n in cnodes])
#     print(node["name"], osd_counts, osd_health)
    ret = {
        "name": node["name"],
        "type": node["type"],
        "children": cnodes,
        "osd_counts": osd_counts,
        "osd_health": osd_health,
        "data": node
    }
    return ret

In [34]:
def getTree(nodes):
    roots = getRoots(nodes);
    cnodes = [getTree_(i, nodes) for i in roots];
    return {
        "name": "_root",
        "children": cnodes
    };


def getTree_(i, nodes):
    node = nodes[i]

    if node["type"] == "osd":
        return getOSDNode(node)
    
    cnodes = [getTree_(str(n), nodes) for n in node["children"]]
    ret = getInternalNode(node, cnodes)
    
    return ret

In [37]:
import os

rawdata_dir = "rawdata"
product_dir = "product"

try:
    os.mkdir(product_dir)
    print("Creat product directory")
except:
    print("Exist product directory")


# Find all json files in the rawdata directory
jsonFiles = glob.glob("{0}/*.json".format(rawdata_dir))
for fs in jsonFiles:
    # Construct the path to store data
    basename = os.path.basename(fs)
    fdname = os.path.splitext(basename)[0]
    fdpath = product_dir + "/" + fdname + ".product.json"    
    with open(fs) as f:
        print("Produce {0}".format(fs))
        data = json.load(f)
        t = getTree(indexableNodes(data["nodes"]))
        with open(fdpath, "w+") as fd:

#             json.dump(t, fd)
            json.dump(t, fd, indent=2)
            print("  - Write to {0}".format(fdpath))

Exist product directory
Produce rawdata/jim.json
  - Write to product/jim.product.json
Produce rawdata/erin.json
  - Write to product/erin.product.json
Produce rawdata/kelly.json
  - Write to product/kelly.product.json
Produce rawdata/beesly.json
  - Write to product/beesly.product.json
